In [1]:
import csv
import pandas as pd

## Load data

In [2]:
hostipitality_customers = pd.read_csv('data/HospitalityCustomers.txt')
reservations = pd.read_csv('data/Reservations.txt')
restaurant_locations = pd.read_csv('data/Restaurant_Locations.txt')
survey_contacts = pd.read_csv('data/SurveyContacts.txt')
surveys = pd.read_csv('data/Surveys.txt')

c:\users\tkyipicard\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
c:\users\tkyipicard\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## delete all the empty columns

In [3]:
surveys=surveys.drop(columns=['AttributeA', 'AttributeB', 'AttributeC', 'AttributeD', 'AttributeE'])
survey_contacts=survey_contacts.drop(columns=['AttributeA', 'AttributeB', 'AttributeC', 'AttributeD'])
restaurant_locations=restaurant_locations.drop(columns=['AttributeA', 'AttributeB', 'AttributeC', 'AttributeD'])
reservations=reservations.drop(columns=['AttributeA', 'AttributeB', 'AttributeC', 'AttributeD', 'AttributeE'])
hostipitality_customers = hostipitality_customers.drop(columns=['AttributeA', 'AttributeB', 'AttributeC', 'AttributeD'])

## Combining dataframes

In [4]:
# merge reservations and hospitality_customers on CustomerID.
hospitality_reservations = hostipitality_customers.merge(reservations, how='inner', on='CustomerID')

# locationID of hospitality_reservations avec restaurant_locations
hos_res_loc = hospitality_reservations.merge(restaurant_locations, how='inner', on="LocationID")

#merging surveys and surveys_contacts
surveys_survey_contacts = survey_contacts.merge(surveys, how='inner', on='SurveyContactID')

In [26]:
print(reservations.shape)
print(hostipitality_customers.shape)
print(hospitality_reservations.shape)
print(restaurant_locations.shape)

(1566569, 10)
(1000006, 14)
(1566569, 23)
(931, 9)


## rename column names

In [5]:
#rename columns
import re
hos_res_loc = hos_res_loc.rename(columns=lambda x: re.sub('_x','',x))
hos_res_loc = hos_res_loc.rename(columns=lambda y: re.sub('_y', '_restaurant', y))

In [6]:
#1566569 rows × 31 columns without dropna
#418 rows × 31 columns with dropna

## Column names

In [51]:
reservations.columns

Index(['ReservationID', 'CustomerID', 'LocationID', 'Source',
       'ReservationDate', 'CheckOutDate', 'NumberOfGuests', 'TotalPrice',
       'Subject', 'Description'],
      dtype='object')

In [126]:
surveys_survey_contacts.columns

Index(['SurveyContactID', 'Name', 'Email', 'DOB', 'Gender', 'StreetAddress',
       'City', 'StateProvince', 'Country', 'PostalCode', 'PhoneNumber',
       'ExtraActivity', 'SurveyID', 'SurveyDate', 'SurveyQuestion', 'Response',
       'Subject', 'Description'],
      dtype='object')

## column Value intersections

No intersections between CustomerID and SurveyContactID

But EmailID and Email have intersections

In [128]:
#check the number of phone number matches between the two columns
hos_res_loc['PhoneNumber'].isin(hos_res_loc['Phone']).value_counts()#

False    1512944
True       53625
Name: PhoneNumber, dtype: int64

In [135]:
#check the number of phone number matches between the two columns
hos_res_loc['Phone'].isin(hos_res_loc['PhoneNumber']).value_counts()#

False    1489065
True       77504
Name: Phone, dtype: int64

In [16]:
hos_res_loc.EmailId.isin(surveys_survey_contacts.Email).value_counts()

True     1080064
False     486505
Name: EmailId, dtype: int64

In [17]:
surveys_survey_contacts.Email.isin(hos_res_loc.EmailId).value_counts()

True     1092383
False      57499
Name: Email, dtype: int64

In [12]:
hos_res_loc.CustomerID.isin(surveys_survey_contacts.SurveyContactID).value_counts()

False    1566569
Name: CustomerID, dtype: int64

In [33]:
pd.crosstab(surveys_survey_contacts.Gender, surveys_survey_contacts.Response)

Response,5,6,7,8,9,10
Gender,,,,,,
Female,78611,56865,44911,24827,235406,135521
Male,78464,56517,44746,24772,234146,135096


## Merge hos_res_loc and surveys_survey_contacts on EmailId

In [19]:
#rename hos_res_loc.EmailId as Email
hos_res_loc = hos_res_loc.rename(columns={"EmailId":"Email"})

In [47]:
hos_res_loc.columns.intersection(surveys_survey_contacts.columns)

Index(['Name', 'City', 'Country', 'Email', 'Gender', 'DOB', 'ExtraActivity',
       'Subject', 'Description', 'PhoneNumber'],
      dtype='object')

In [49]:
print(hos_res_loc[['Name', 'City', 'Country', 'Email', 'Gender', 'DOB', 'ExtraActivity',
       'Subject', 'Description', 'PhoneNumber']].dtypes)

Name             object
City             object
Country          object
Email            object
Gender           object
DOB              object
ExtraActivity    object
Subject          object
Description      object
PhoneNumber      object
dtype: object


In [50]:
print(surveys_survey_contacts[['Name', 'City', 'Country', 'Email', 'Gender', 'DOB', 'ExtraActivity',
       'Subject', 'Description', 'PhoneNumber']].dtypes)

Name             object
City             object
Country          object
Email            object
Gender           object
DOB              object
ExtraActivity    object
Subject          object
Description      object
PhoneNumber      object
dtype: object


In [52]:
# data_merged = surveys_survey_contacts.merge(hos_res_loc, how='inner', on='Email')

## resturant and customer are in the same area or not

In [174]:
hos_res_loc.City.isin(hos_res_loc.City_restaurant).value_counts()

True     1361415
False     205154
Name: City, dtype: int64

## null values

In [141]:
pd.isnull(hos_res_loc).any()

Name                   True
Address                True
City                  False
State                 False
Country               False
Zip                   False
EmailId                True
Gender                False
DOB                   False
Phone                  True
SSN                   False
Individual            False
ExtraActivity          True
CustomerID            False
ReservationID         False
LocationID            False
Source                False
ReservationDate       False
CheckOutDate          False
NumberOfGuests        False
TotalPrice            False
Subject               False
Description           False
Name_restaurant        True
Address_restaurant     True
City_restaurant       False
State_restaurant      False
Zip_restaurant        False
Country_restaurant    False
PhoneNumber            True
NoOfTables            False
dtype: bool

In [142]:
pd.isnull(surveys_survey_contacts).any()

SurveyContactID    False
Name                True
Email               True
DOB                False
Gender             False
StreetAddress       True
City               False
StateProvince      False
Country            False
PostalCode         False
PhoneNumber         True
ExtraActivity       True
SurveyID           False
SurveyDate         False
SurveyQuestion     False
Response           False
Subject            False
Description        False
dtype: bool

## Delete further un desired columns

## Feature selection